# F1 и каппа оценка классификации

1. Разделить выборку на обуч./пров. в соотн. 80/20
2. Для обучения вычислить среднее значение для веса, роста, индекса массы тела и возраста - для каждого из принятых решений. Предсказать оценку скоринга по близости данных средним значениям.
3. Проверить качество предсказаний через F1-метрику и матрицу неточностей.

## Подключение библиотек

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, cohen_kappa_score

## Загрузка данных

In [2]:
data = pd.read_csv("../data/prudential/train.csv.gz")
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


## Разделить данные на обучающие и проверочные, 80/20

In [4]:
data_train, data_test = train_test_split(data, test_size=0.2)
print(data_train.head())

          Id  Product_Info_1 Product_Info_2  Product_Info_3  Product_Info_4  \
50471  67211               1             D3              26        1.000000   
45159  60059               1             D3              26        0.487179   
40949  54404               1             A8              26        1.000000   
12752  16930               1             D3              26        0.076923   
31174  41391               1             C3              26        0.230769   

       Product_Info_5  Product_Info_6  Product_Info_7   Ins_Age        Ht  \
50471               2               3               1  0.223881  0.745455   
45159               2               3               1  0.238806  0.781818   
40949               2               3               1  0.238806  0.709091   
12752               2               3               1  0.074627  0.690909   
31174               2               3               1  0.298507  0.709091   

       ...  Medical_Keyword_40  Medical_Keyword_41  Medical_Ke

## Вычислить медиану для каждой оценки

In [12]:
columns = [
    "Wt",
    "Ht",
    "Ins_Age",
    "BMI",
]

responses = np.arange(1, data["Response"].max() + 1)
clusters = [{} for _ in range(len(responses) + 1)]

for r in responses:
    for c in columns:
        clusters[r][c] = data[data["Response"] == r][c].median()

print(clusters)

[{}, {'Wt': 0.309623431, 'Ht': 0.709090909, 'Ins_Age': 0.52238806, 'BMI': 0.483173533}, {'Wt': 0.330543933, 'Ht': 0.727272727, 'Ins_Age': 0.47761194, 'BMI': 0.510582282}, {'Wt': 0.317991632, 'Ht': 0.727272727, 'Ins_Age': 0.358208955, 'BMI': 0.510582282}, {'Wt': 0.257322176, 'Ht': 0.709090909, 'Ins_Age': 0.328358209, 'BMI': 0.4196545055}, {'Wt': 0.351464435, 'Ht': 0.709090909, 'Ins_Age': 0.402985075, 'BMI': 0.5918224864999999}, {'Wt': 0.309623431, 'Ht': 0.727272727, 'Ins_Age': 0.432835821, 'BMI': 0.489742491}, {'Wt': 0.290794979, 'Ht': 0.709090909, 'Ins_Age': 0.447761194, 'BMI': 0.470772683}, {'Wt': 0.236401674, 'Ht': 0.690909091, 'Ins_Age': 0.313432836, 'BMI': 0.394874563}]


## Выполнить предсказание оценки скоринга на основе медианы

Используем евклидово расстояние:
$$ D=\sqrt{\sum(a_i - C_i)^2} $$


In [13]:
def calc_model(x, clusters, columns, data):
    D_min = 10000000
    target = 0
    for i, cluster in enumerate(clusters):
        if cluster:
            D = 0
            for c in columns:
                D += (cluster[c] - x[c])**2
            D = np.sqrt(D)
            if D < D_min:
                target = i
                D_min = D
    x["target"] = target
    x["random"] = int(np.random.uniform(1, 8.01, 1)[0])
    x["sample"] = data.sample(1)["Response"].values[0]
    x["all8"] = 8
    return x

In [15]:
data_test = data_test.apply(calc_model, axis=1, result_type="expand", args=(clusters, columns, data))
print(data_test.head(20))

          Id  Product_Info_1 Product_Info_2  Product_Info_3  Product_Info_4  \
31712  42095               1             A2              26        0.230769   
23574  31443               1             D3              26        0.333333   
32416  43005               1             A8              26        0.076923   
18356  24492               1             D4              26        0.384615   
10228  13595               1             A5              26        1.000000   
56008  74609               1             A5              26        0.076923   
52129  69406               1             D4              26        0.487179   
46847  62340               1             D4              26        0.333333   
7736   10298               1             D2              26        0.128205   
59045  78734               1             D3              26        0.230769   
55262  73587               1             C4              26        1.000000   
26029  34657               2             A1         

## Оценка качества модели: F1

In [16]:
print(f"Случайный выбор: {f1_score(data_test['random'], data_test['Response'], average='weighted')}")
print(f"Выбор по частоте: {f1_score(data_test['sample'], data_test['Response'], average='weighted')}")
print(f"Кластеризация: {f1_score(data_test['target'], data_test['Response'], average='weighted')}")
print(f"Самый популярный: {f1_score(data_test['all8'], data_test['Response'], average='weighted')}")

Случайный выбор: 0.10298054043901518
Выбор по частоте: 0.19152672816479946
Кластеризация: 0.2767571635554511
Самый популярный: 0.5006627532664268


## Матрица неточностей

In [17]:
print(confusion_matrix(data_test["target"], data_test["Response"]))

[[ 512  444   36   45  286  655  540  812]
 [  87   95   18    7   72  192  120   64]
 [  92  108   40   39  153  389  268  188]
 [  62   54   15   48   34  214  169  488]
 [ 216  361   48    2  458  271   33    7]
 [  26   31    1    2    7   64   39   72]
 [  69   69   11   24   26  114  139  374]
 [ 157  155   32  112   74  318  258 1961]]


## Квадратный коэффициент Каппа Коэна

In [18]:
print(cohen_kappa_score(data_test["target"], data_test["Response"], weights="quadratic"))
print(cohen_kappa_score(data_test["all8"], data_test["Response"], weights="quadratic"))

0.197895147582815
0.0
